In [1]:
! python3 -m pip install --upgrade pip
! python3 -m pip install --upgrade pylangacq

     |████████████████████████████████| 1.5MB 5.6MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 66 kB 1.9 MB/s 


In [2]:

# Python packages
import io
import os
import pandas as pd
import random
from os import listdir
from os.path import isfile, join
from os import path

# Cloud storage
from google.cloud import storage

# .cha file reader
import pylangacq

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = '/content/drive/My Drive/Berkeley/W210/Capstone/ADReSS-IS2020-data/train'
text_path_cc = data_path + '/transcription/cc/'

## Define all the file path variables

In [5]:
# TODO make an input from user
# data_path = '/tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train'
data_path = '/content/drive/My Drive/Berkeley/W210/Capstone/ADReSS-IS2020-data/train'

audio_path_cc = data_path + '/Full_wave_enhanced_audio/cc/'
audio_path_cd = data_path + '/Full_wave_enhanced_audio/cd/'


text_path_cc = data_path + '/transcription/cc/'
text_path_cd = data_path + '/transcription/cd/'

lang_ = 'en-US'



## Convert .cha transcriptions to text file by selecting only the participants part of the text without any shuffles

In [8]:
def convertChaToTextFormatWithoutShuffle(input_file_dir, output_file_dir):
  df_id_txt = pd.DataFrame(columns=['ID', 'TEXT'])

  # If the output directory doesn't exists, create one
  if not os.path.exists(output_file_dir):
      os.makedirs(output_file_dir)    

  onlyfiles = [f for f in sorted(listdir(input_file_dir)) if isfile(join(input_file_dir, f))]
  # print(len(onlyfiles))

  for filename in onlyfiles:
    file_path = input_file_dir + "/" + filename
    # print(file_path)
    reader = pylangacq.read_chat(file_path)
    reader.participants()
    sentence_words = reader.words(participants="PAR", by_utterances=True)
    file_name_prefix = filename.partition('.')[0]

    final_text = ''
    for sentence in sentence_words:
      for word in sentence:
        final_text = final_text + ' ' + word

    final_text_modified = final_text.replace(". and", "and")
    final_text_modified = final_text_modified.replace(" . ", ".")

    output_text_file = file_name_prefix + ".txt"
    output_text_file_path = output_file_dir + output_text_file
    fh = open(output_text_file_path, "w+") 
    fh.write(final_text)
    df_id_txt.loc[len(df_id_txt.index)] = [file_name_prefix, final_text]

  return df_id_txt

df_id_txt_cc_orig = convertChaToTextFormatWithoutShuffle(text_path_cc, data_path + "/transcription_original_text/")
df_id_txt_cd_orig = convertChaToTextFormatWithoutShuffle(text_path_cd, data_path + "/transcription_original_text/")
df_id_txt_orig = pd.DataFrame()
df_id_txt_orig = df_id_txt_orig.append(df_id_txt_cc_orig)
df_id_txt_orig = df_id_txt_orig.append(df_id_txt_cd_orig)
df_id_txt_orig = df_id_txt_orig[["ID", "TEXT"]]
df_id_txt_orig = df_id_txt_orig.reset_index()

df_id_txt_orig.tail()

,index,ID,TEXT
103,49,S150,"well the boy on the stool is falling , reachi..."
104,50,S151,the boy and the girl are playing and he's gon...
105,51,S153,oh you want me to on that ? oh okay looks lik...
106,52,S154,you want me to tell you ? okay the boy's gett...
107,53,S156,mhm . well this one is in the cookie jar . an...


## Convert .cha transcriptions to text file by selecting only the participants part of the text

In [11]:
def convertChaToTextFormatWithShuffling(input_file_dir, output_file_dir, combination_count = 1):
  df_id_txt = pd.DataFrame(columns=['ID', 'TEXT'])

  # If the output directory doesn't exists, create one
  if not os.path.exists(output_file_dir):
      os.makedirs(output_file_dir)    

  onlyfiles = [f for f in sorted(listdir(input_file_dir)) if isfile(join(input_file_dir, f))]
  # print(len(onlyfiles))

  for filename in onlyfiles:
    file_path = input_file_dir + "/" + filename
    # print(file_path)
    reader = pylangacq.read_chat(file_path)
    reader.participants()
    sentence_words = reader.words(participants="PAR", by_utterances=True)
    file_name_prefix = filename.partition('.')[0]

    max_combination_count = combination_count
    if len(sentence_words) < max_combination_count:
      max_combination_count = len(sentence_words)

    # create combinations of sentences using random shuffling of the list
    for count in range(0, max_combination_count):
      if count > 0:
        random.shuffle(sentence_words)
        
      final_text = ''
      for sentence in sentence_words:
        for word in sentence:
          final_text = final_text + ' ' + word

      final_text_modified = final_text.replace(". and", "and")
      final_text_modified = final_text_modified.replace(" . ", ".")

      output_text_file = file_name_prefix + "_" + str(count) + ".txt"
      output_text_file_path = output_file_dir + output_text_file
      fh = open(output_text_file_path, "w+") 
      fh.write(final_text)
      df_id_txt.loc[len(df_id_txt.index)] = [file_name_prefix, final_text]

  return df_id_txt

df_id_txt_cc = convertChaToTextFormatWithShuffling(text_path_cc, data_path + "/transcription_shuffled_text/", combination_count=20)
df_id_txt_cd = convertChaToTextFormatWithShuffling(text_path_cd, data_path + "/transcription_shuffled_text/", combination_count=20)
df_id_txt = pd.DataFrame()
df_id_txt = df_id_txt.append(df_id_txt_cc)
df_id_txt = df_id_txt.append(df_id_txt_cd)
df_id_txt = df_id_txt[["ID", "TEXT"]]
df_id_txt = df_id_txt.reset_index()

df_id_txt.tail()


,index,ID,TEXT
1395,677,S156,he was climbing and the stool tipped over . t...
1396,678,S156,and she's watching . he was climbing and the ...
1397,679,S156,oh ‡ no . well this one is in the cookie jar ...
1398,680,S156,this boy tried to get in the cookie jar . oh ...
1399,681,S156,and she's watching . mhm . well this one is i...


# Merge meta data to txt data by ID and update the MMSE score

In [12]:
meta_cc = pd.read_csv(data_path + '/cc_meta_data.txt', sep=";", header=0, 
                      names = ['ID', 'Age', 'Gender', 'MMSE'])
meta_cd = pd.read_csv(data_path + '/cd_meta_data.txt', sep=";", header=0, 
                      names = ['ID', 'Age', 'Gender', 'MMSE'])

meta = meta_cc.assign(Group = 'cc').append(meta_cd.assign(Group = 'cd')).reset_index()

meta['ID'] = meta['ID'].str.strip()

meta

,index,ID,Age,Gender,MMSE,Group
0,0,S001,74,male,NA,cc
1,1,S002,62,female,30,cc
2,2,S003,69,female,29,cc
3,3,S004,71,female,30,cc
4,4,S005,74,female,30,cc
...,...,...,...,...,...,...
103,49,S150,58,male,20,cd
104,50,S151,72,male,24,cd
105,51,S153,68,female,12,cd
106,52,S154,65,female,20,cd


In [13]:
sum(meta.MMSE == ' NA')
meta.MMSE = pd.to_numeric(meta.MMSE.replace(' NA', 30))

In [14]:
merged_data = pd.merge(meta, df_id_txt, on = "ID", how = "inner")
merged_data

,index_x,ID,Age,Gender,MMSE,Group,index_y,TEXT
0,0,S001,74,male,30,cc,0,well there's a mother standing there washing ...
1,0,S001,74,male,30,cc,1,he's standing up there in the cupboard taking...
2,0,S001,74,male,30,cc,2,and outside the window there's a curved walk ...
3,0,S001,74,male,30,cc,3,and the window's open . the kids are somewher...
4,0,S001,74,male,30,cc,4,and there are two cups and a saucer on the si...
...,...,...,...,...,...,...,...,...
1395,53,S156,71,female,13,cd,677,he was climbing and the stool tipped over . t...
1396,53,S156,71,female,13,cd,678,and she's watching . he was climbing and the ...
1397,53,S156,71,female,13,cd,679,oh ‡ no . well this one is in the cookie jar ...
1398,53,S156,71,female,13,cd,680,this boy tried to get in the cookie jar . oh ...


In [15]:

filtered_merged_data = merged_data[merged_data['TEXT'].apply(lambda x: len(x) > 0)]
filtered_merged_data = filtered_merged_data[["ID", "Age", "Gender", "TEXT", "MMSE", "Group"]]
filtered_merged_data


,ID,Age,Gender,TEXT,MMSE,Group
0,S001,74,male,well there's a mother standing there washing ...,30,cc
1,S001,74,male,he's standing up there in the cupboard taking...,30,cc
2,S001,74,male,and outside the window there's a curved walk ...,30,cc
3,S001,74,male,and the window's open . the kids are somewher...,30,cc
4,S001,74,male,and there are two cups and a saucer on the si...,30,cc
...,...,...,...,...,...,...
1395,S156,71,female,he was climbing and the stool tipped over . t...,13,cd
1396,S156,71,female,and she's watching . he was climbing and the ...,13,cd
1397,S156,71,female,oh ‡ no . well this one is in the cookie jar ...,13,cd
1398,S156,71,female,this boy tried to get in the cookie jar . oh ...,13,cd


In [16]:
transcription_combination_txt_file_path = data_path + "/transcription_shuffled_text/" + "transcription_combination_id_txt_mmse.csv"

In [17]:
filtered_merged_data.to_csv(transcription_combination_txt_file_path)

In [18]:
filtered_merged_data = pd.read_csv(transcription_combination_txt_file_path)
filtered_merged_data.tail()

,Unnamed: 0,ID,Age,Gender,TEXT,MMSE,Group
1395,1395,S156,71,female,he was climbing and the stool tipped over . t...,13,cd
1396,1396,S156,71,female,and she's watching . he was climbing and the ...,13,cd
1397,1397,S156,71,female,oh ‡ no . well this one is in the cookie jar ...,13,cd
1398,1398,S156,71,female,this boy tried to get in the cookie jar . oh ...,13,cd
1399,1399,S156,71,female,and she's watching . mhm . well this one is i...,13,cd


In [19]:
merged_data_orig = pd.merge(meta, df_id_txt_orig, on = "ID", how = "inner")
merged_data_orig

,index_x,ID,Age,Gender,MMSE,Group,index_y,TEXT
0,0,S001,74,male,30,cc,0,well there's a mother standing there washing ...
1,1,S002,62,female,30,cc,1,somebody's getting cookies out_of the cookie ...
2,2,S003,69,female,29,cc,2,okay . there's a little boy and he's standing...
3,3,S004,71,female,30,cc,3,are you ready ? well the sink is overflowing ...
4,4,S005,74,female,30,cc,4,okay . the mother's washing the dishes and th...
...,...,...,...,...,...,...,...,...
103,49,S150,58,male,20,cd,49,"well the boy on the stool is falling , reachi..."
104,50,S151,72,male,24,cd,50,the boy and the girl are playing and he's gon...
105,51,S153,68,female,12,cd,51,oh you want me to on that ? oh okay looks lik...
106,52,S154,65,female,20,cd,52,you want me to tell you ? okay the boy's gett...


In [20]:
filtered_merged_data_orig = merged_data_orig[merged_data_orig['TEXT'].apply(lambda x: len(x) > 0)]
filtered_merged_data_orig = filtered_merged_data_orig[["ID", "Age", "Gender", "TEXT", "MMSE", "Group"]]
filtered_merged_data_orig


,ID,Age,Gender,TEXT,MMSE,Group
0,S001,74,male,well there's a mother standing there washing ...,30,cc
1,S002,62,female,somebody's getting cookies out_of the cookie ...,30,cc
2,S003,69,female,okay . there's a little boy and he's standing...,29,cc
3,S004,71,female,are you ready ? well the sink is overflowing ...,30,cc
4,S005,74,female,okay . the mother's washing the dishes and th...,30,cc
...,...,...,...,...,...,...
103,S150,58,male,"well the boy on the stool is falling , reachi...",20,cd
104,S151,72,male,the boy and the girl are playing and he's gon...,24,cd
105,S153,68,female,oh you want me to on that ? oh okay looks lik...,12,cd
106,S154,65,female,you want me to tell you ? okay the boy's gett...,20,cd


In [21]:
transcription_combination_txt_file_path_orig = data_path + "/transcription_original_text/" + "transcription_original_id_txt_mmse.csv"

In [22]:
filtered_merged_data_orig.to_csv(transcription_combination_txt_file_path_orig)

In [23]:
filtered_merged_data_orig = pd.read_csv(transcription_combination_txt_file_path_orig)
filtered_merged_data_orig.tail()

,Unnamed: 0,ID,Age,Gender,TEXT,MMSE,Group
103,103,S150,58,male,"well the boy on the stool is falling , reachi...",20,cd
104,104,S151,72,male,the boy and the girl are playing and he's gon...,24,cd
105,105,S153,68,female,oh you want me to on that ? oh okay looks lik...,12,cd
106,106,S154,65,female,you want me to tell you ? okay the boy's gett...,20,cd
107,107,S156,71,female,mhm . well this one is in the cookie jar . an...,13,cd
